In [5]:
import pandas as pd
from pymongo import MongoClient
from datetime import datetime, timedelta

server = "192.168.2.68"
port = "27018"
IOTS_READING = ["BuildingLeftSide", "iots_reading_left_side"]
client = MongoClient(server + ':' + port)

now = datetime.now()
start = now - timedelta(days=6, hours=now.hour, minutes=now.minute)
end = start + timedelta(days=1)

historic = list(client[IOTS_READING[0]][IOTS_READING[1]].find({'datetime': {'$gt': start, '$lt': end}}))
client.close()

In [6]:
forecast = pd.DataFrame(historic)
total = forecast.drop(["_id"], axis=1)

total = total.dropna()

total = total.values.tolist()
total_power = []

for row in total:
    iots = row[0]
    date = row[1]
    consumption = 0
    for iot in iots:
        for value in iot['values']:
            if 'values' in value:
                if value['type'] == 'power':
                    consumption += value['values']

    total_power.append([date, consumption])

forecast = pd.DataFrame(total_power, columns=['datetime', 'consumption'])

In [7]:
forecast['datetime'] = pd.to_datetime(forecast['datetime'], format='%Y-%m-%d %H:%M:%S', dayfirst=True)
forecast.set_index("datetime", inplace=True)


In [8]:
forecast.resample('1H').mean()

,consumption
datetime,
2023-12-12 10:00:00,1700.000000
2023-12-12 11:00:00,1701.777778
2023-12-12 12:00:00,1701.077778
2023-12-12 13:00:00,1706.933333
2023-12-12 14:00:00,1708.000000
2023-12-12 15:00:00,1708.000000
2023-12-12 16:00:00,1708.000000
2023-12-12 17:00:00,1708.000000
2023-12-12 18:00:00,1709.997222


In [ ]:
len(forecast)

In [ ]:
import pandas as pd
from database.BuildingRepository import BuildingRepository
from pandas.api.types import is_numeric_dtype
from datetime import datetime, timedelta

building_repo = BuildingRepository()
df = pd.DataFrame(building_repo.get_power_historic(datetime.now() - timedelta(days=30)))
df = df.drop("_id", axis=1)
df['datetime'] = pd.to_datetime(df['datetime'])

df.set_index("datetime", inplace=True)
df = df.resample('15T').mean()
df["datetime"] = df.index

df['totalpower'] = pd.to_numeric(df['totalpower'], errors='coerce')
df['Month'] = df['datetime'].dt.month
df['Day'] = df['datetime'].dt.day
df['Hour'] = df['datetime'].dt.hour
df['Weekday'] = df['datetime'].dt.weekday

df.rename(columns={'totalpower': 'Consumption'}, inplace=True)
df.drop(['datetime', 'totalgeneration'], axis=1, inplace=True)

df = df.dropna()
df['Consumption-1'] = df['Consumption'].shift(1)
df.loc[df['Day'] != df['Day'].shift(1), 'Consumption-1'] = 0
df['Consumption-2'] = df['Consumption'].shift(2)
df.loc[df['Day'] != df['Day'].shift(2), 'Consumption-2'] = 0
#df = df[['Month', 'Day', 'Hour', 'Consumption-1', 'Consumption-2', 'Consumption']]
df = df[['Month', 'Day', 'Hour', 'Weekday', 'Consumption-1', 'Consumption-2', 'Consumption']]
df

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Data preprocessing - Scaling using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df)
pred_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaled = pred_scaler.fit_transform(np.array(df['Consumption']).reshape(-1, 1))

# split into train and test sets
train_size = int(len(df_scaled) * 0.8)
test_size = len(df_scaled) - train_size
train, test = df_scaled[0:train_size, :], df_scaled[train_size:len(df_scaled), :]
print("Len train", len(train), "Len test", len(test))

In [ ]:
import joblib

# Assuming you have a scaler object named 'scaler'
joblib.dump(scaler, 'training_1/scaler.pkl')
joblib.dump(pred_scaler, 'training_1/pred_scaler.pkl')

In [ ]:
import numpy as np


# Prepare data for LSTM (sequence-to-sequence)
def prepare_data(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps - 2):
        X.append(data[i:i + time_steps, :-1])  # Exclude the last column (Consumption)
        y.append(data[i + time_steps, -1])  # The last column (Consumption) is the target
    return np.array(X), np.array(y)


timesteps = 1
X_train, y_train = prepare_data(train, timesteps)
X_test, y_test = prepare_data(test, timesteps)

print("X_train shape", X_train.shape, "timesteps", timesteps)
print("X_test shape", X_test.shape, "timesteps", timesteps)

# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], timesteps, X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], timesteps, X_test.shape[2]))

print("X_train shape", X_train.shape, "y_train shape", y_train.shape)
print("X_test shape", X_test.shape, "y_test shape", y_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import tensorflow as tf


def create_model():
    # LSTM model
    model = Sequential()
    print(timesteps, X_train.shape[2])
    model.add(LSTM(units=50, input_shape=(timesteps, X_train.shape[2])))
    model.add(Dense(units=1))  # Use the original number of features as the output units
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

    return model


model = create_model()

# Display the model's architecture
model.summary()

In [ ]:
import os

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1, callbacks=[cp_callback])

In [ ]:
y_test

In [ ]:
# Evaluate the model
loss, mae, mse = model.evaluate(X_test, y_test, verbose=2)
print("Trained model, mae: {:5.2f}%, mse: {:5.2f}%".format(mae, mse))

model.save('saved_model/consumption_forecast')

y_train_inv = pred_scaler.inverse_transform(y_train.reshape(-1, 1))
y_test_inv = pred_scaler.inverse_transform(y_test.reshape(-1, 1))

# Make predictions on train and test sets
y_train_pred = model.predict(X_train)
y_train_pred = pred_scaler.inverse_transform(y_train_pred)

y_test_pred = model.predict(X_test)
y_test_pred = pred_scaler.inverse_transform(y_test_pred)

# Display the original DataFrame and the predicted values after inverse scaling for train and test sets
print("Original DataFrame:")
print("y_train_inv.shape", y_train_inv.shape, "y_train_pred.shape", y_train_pred.shape)

In [ ]:
model.save_weights('./checkpoints/my_checkpoint')

In [ ]:
model_test = create_model()

loss, mae, mse = model_test.evaluate(X_test, y_test, verbose=2)
print("Trained model, mae: {:5.2f}%, mse: {:5.2f}%".format(mae, mse))

In [ ]:
# Restore the weights
model_test.load_weights('./checkpoints/my_checkpoint')

# Re-evaluate the model
loss, mae, mse = model_test.evaluate(X_test, y_test, verbose=2)
print("Trained model, mae: {:5.2f}%, mse: {:5.2f}%".format(mae, mse))

In [ ]:
new_model = tf.keras.models.load_model('saved_model/consumption_forecast')

# Check its architecture
new_model.summary()

# Evaluate the restored model
loss, mae, mse = new_model.evaluate(X_test, y_test, verbose=2)
print("Trained model, mae: {:5.2f}%, mse: {:5.2f}%".format(mae, mse))

print("New Model Shape", new_model.predict(X_test).shape)

In [ ]:
import matplotlib.pyplot as plt

# Plot predictions vs actual
plt.figure(figsize=(10, 6))
plt.plot(y_train_inv, label='Actual (Train)', color='blue')
plt.plot(y_train_pred, label='Pred (Train)', linestyle='dashed', color='red')

plt.xlabel('Time Steps')
plt.ylabel('Consumption')
plt.legend()
plt.show()

# Plot predictions vs actual
plt.figure(figsize=(10, 6))

plt.plot(y_test_inv, label='Actual (Test)', color='green')
plt.plot(y_test_pred, label='Pred (Test)', linestyle='dashed', color='orange')

plt.xlabel('Time Steps')
plt.ylabel('Consumption')
plt.legend()
plt.show()


## Run modelwith saved weights
Get Data

In [ ]:
import pandas as pd
from database.BuildingRepository import BuildingRepository
from datetime import datetime, timedelta

building_repo = BuildingRepository()
df_test = pd.DataFrame(building_repo.get_power_historic(datetime.now() - timedelta(hours=24)))
df_test = df_test.drop("_id", axis=1)

df_test['datetime'] = pd.to_datetime(df_test['datetime'])
# Calculate the datetime for 24 hours ago from the current time
current_time = df_test['datetime'].max()
twenty_four_hours_ago = current_time - timedelta(hours=24)

# Filter the DataFrame to get only the rows for the last 24 hours
last_24_hours_data = df_test[df_test['datetime'] >= twenty_four_hours_ago]

last_24_hours_data['totalpower'] = pd.to_numeric(last_24_hours_data['totalpower'], errors='coerce')
last_24_hours_data['Month'] = last_24_hours_data['datetime'].dt.month
last_24_hours_data['Day'] = last_24_hours_data['datetime'].dt.day
last_24_hours_data['Hour'] = last_24_hours_data['datetime'].dt.hour
last_24_hours_data['Minute'] = last_24_hours_data['datetime'].dt.minute

last_24_hours_data.rename(columns={'totalpower': 'Consumption'}, inplace=True)

last_24_hours_data['datetime'] = pd.to_datetime(last_24_hours_data['datetime'], format='%Y-%m-%d %H:%M:%S',
                                                dayfirst=True)
last_24_hours_data.set_index("datetime", inplace=True)
last_24_hours_data = last_24_hours_data.resample('15T').mean()

last_24_hours_data.drop(['totalgeneration'], axis=1, inplace=True)

last_24_hours_data = last_24_hours_data.dropna()
last_24_hours_data['Consumption-1'] = last_24_hours_data['Consumption'].shift(1)
last_24_hours_data.loc[last_24_hours_data['Day'] != last_24_hours_data['Day'].shift(1), 'Consumption-1'] = 0
last_24_hours_data['Consumption-2'] = last_24_hours_data['Consumption'].shift(2)
last_24_hours_data.loc[last_24_hours_data['Day'] != last_24_hours_data['Day'].shift(2), 'Consumption-2'] = 0
#last_24_hours_data = last_24_hours_data[['Month', 'Day', 'Hour', 'Consumption-1', 'Consumption-2', 'Consumption']]
last_24_hours_data = last_24_hours_data[
    ['Month', 'Day', 'Hour', 'Minute', 'Consumption-1', 'Consumption-2', 'Consumption']]
last_24_hours_data

Load model

In [ ]:
import joblib

# Load the scaler
scaler_test = joblib.load('training_1/scaler.pkl')
pred_scaler_test = joblib.load('training_1/pred_scaler.pkl')

df_scaled = scaler_test.fit_transform(last_24_hours_data)

print("Len pred", int(len(df_scaled)))

In [ ]:
import numpy as np

timesteps = 1


def prepare_data(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps - 2):
        X.append(data[i:i + time_steps, :-1])  # Exclude the last column (Consumption)
        y.append(data[i + time_steps, -1])  # The last column (Consumption) is the target
    return np.array(X), np.array(y)


X_pred, y = prepare_data(df_scaled, timesteps)

print("X_pred shape", X_pred.shape, "timesteps", timesteps)

# reshape input to be [samples, time steps, features]
X_pred = np.reshape(X_pred, (X_pred.shape[0], timesteps, X_pred.shape[2]))

print("X_pred shape", X_pred.shape, "y_pred shape", y.shape)

### load model

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('saved_model/consumption_forecast')
y_pred = model.predict(X_pred)
y_pred = pred_scaler_test.inverse_transform(y_pred)
y = pred_scaler_test.inverse_transform(y.reshape(-1, 1))

In [ ]:
import matplotlib.pyplot as plt

# Plot predictions vs actual
plt.figure(figsize=(10, 6))
plt.plot(y, label='Actual', color='blue')
plt.plot(y_pred, label='Pred', linestyle='dashed', color='red')

plt.xlabel('Time Steps')
plt.ylabel('Consumption')
plt.legend()
plt.show()


In [ ]:
last = scaler_test.inverse_transform(df_scaled[:-3])

df = pd.DataFrame(last, columns=['Month', 'Day', 'Hour', 'Minute', 'Consumption-1', 'Consumption-2', 'Consumption'])
df['Prediction'] = y_pred
df['datetime'] = pd.to_datetime(df[['Month', 'Day', 'Hour', 'Minute']].assign(Year=2023))

# Drop the separate columns if needed
df.drop(['Month', 'Day', 'Hour', 'Minute'], axis=1, inplace=True)

df.set_index("datetime", inplace=True)
df = df.resample('1H').mean()
df["datetime"] = df.index
# Now, 'datetime' contains the combined datetime values
df